In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***
# PCA Modeling
***

Tujuan penelitian ini adalah melakukan PCA Modeling terhadap data emiten saham di sektor aneka industri untuk sub sektor Mesin dan Komponen Industri Q4 2021. 

Menggunakan Data Clustering yang sudah dilakukan sebelumnya. [Clustering Sektor Industri - Aneka Industri 2021](https://www.kaggle.com/code/hajarhanifah/clustering-sektor-industri-hajar-hanifah/edit/run/93274745)

Adapun beberapa step yang akan dilakukan adalah 
1. Import Libraries
2. Exploratory Data Analysis
    - Data Understanding
    - Understanding the Table
    - Load Data
    - Modeling
3. Multinomial logistic regression with scikit-learn
    - Data Preparation
    - Fit The Model
    - Model Validation
    - Probabilitas
 4. Conclution

# PCA Modeling untuk Data Visualization

Dalam aplikasi Machine Learning, PCA Modeling sangat berguna untuk memvisualisasi data yang dimiliki. 

***
# 1. Import Libraries
***

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

***
# 2. Exploratory Data Analysis
***


# 2.1 Data Understanding

Data yang diambil merupakan data statictical financial ratio Q4 2021 dan 2020 untuk seluruh emiten di sektor Industri.

Data di dapatkan dari [IDX - Laporan Statistic Financial Data Ratio](https://www.idx.co.id/data-pasar/laporan-statistik/digital-statistic-beta/financial-data-ratio?q=eyJ5ZWFyIjoiMjAyMSIsIm1vbnRoIjpudWxsLCJxdWFydGVyIjo0LCJ0eXBlIjoieWVhcmx5In0=)


# 2.2 Understanding The Table

* **Emiten :** Pihak yang melakukan penawaran umum, yaitu penawaran efek yang dilakukan oleh emiten untuk menjual efek kepada masyarakat berdasarkan tata cara yang diatur dalam peraturan undang-undang yang berlaku. 
* **Sales Growth :** Kenaikan jumlah penjualan dari tahun ke tahun atau dari waktu ke waktu.
* **ROA (Return of Asset) :** Indikator untuk menunjukkan seberapa untuk sebuah perusahaan dibandingkan dengan total asetnya. 
* **DER (Debt to Equity Ratio) :** Rasio hutang terhadap ekuitas atau rasio keuangan yang membandingkan jumlah hutang dengan ekuitas

Keterangan rasio yang akan digunakan : 
- **DER =** Total Uang / Total Ekuitas
- **Return on Asset =** Laba setelah pajak / Total Asset 
- **Sales Growth =** (penjualan periode 2021 - penjualan periode 2020)/ penjualan periode 2020

# 2.3 Load Data

In [3]:
df = pd.read_csv('../input/ratio-real-estate/Data Financial Ratio Real Estate.csv')
df.head()

In [4]:
df.describe()

In [5]:
display(df)

**Mengganti Null dengan 0**

In [6]:
df.fillna(0, inplace=True)

In [7]:
display(df)

# 2.1 Standardize the Data

PCA dipengaruhi oleh skala / scale, sehingga diperlukan untuk men skalakan fitur dalam data sebelum menerapkan PCA. 
- gunakan StandardScaler untuk membantu menstandarisasi dataset ke unit skala (mean = 0, variance = 1) untuk mengoptimalkan kinerja algoritma dari Machine Learning 
- Untuk melihat efek negatif dari tidak menskalakan dataset, dapat menggunakan sckit-learn

In [8]:
from sklearn.preprocessing import StandardScaler
features = ['DER', 'ROA', 'Sales Growth', 'ROE', 'NPM']
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
y = df.loc[:,['Category']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [9]:
display(df)

In [10]:
x

In [11]:
pca = PCA(n_components=3)

principal_components = pca.fit_transform(x)

new_X = pd.DataFrame(data = principal_components, columns=['PC1','PC2','PC3'])

In [12]:
new_X.head()

In [13]:
per_var = np.round(pca.explained_variance_ratio_ * 100, decimals =1)

label = ['PC' + str(x) for x in range (1, len(per_var)+1)]

plt.bar (x=range(1, len(per_var)+1),height=per_var,tick_label=label)
plt.ylabel('percentage of explained variance')
plt.xlabel('principal components')
plt.title('scree plot')
plt.show()

# 2.2 PCA Projection to 2D

- Data original memiliki 5 kolom (untuk X ) yaitu DER, ROI, Sales Frowth, ROE, NPM
- Pada bagian ini, proyeksikan data asli yaitu 5 dimensional data menjadi 2 dimensi 

In [14]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [15]:
display(principalDf)

In [16]:
finalDf = pd.concat([principalDf, df[['Category']]], axis = 1)

In [17]:
display(finalDf)

# 2.3 Visualize 2D Projection

- Plotting 2 dimensional Data
- Perhatikan pada grafik di bawah ini bahwa kelas-kelas tampak terpisah satu sama lain.

In [18]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['DER', 'ROA', 'Sales Growth']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['Category'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [21]:
pca.explained_variance_ratio_